In [124]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
import os
import scipy.sparse

# Load Data

In [126]:
#**************
# with NaNs
# tX
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-pca-with-dummy-vars-with-NaNs.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-whitened-with-dummy-vars-with-NaNs.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-standardized-with-dummy-vars-with-NaNs.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-standardized-with-dummy-vars-with-NaNs-removed-corr.csv")
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-whitened-using-PCA-with-dummy-vars-with-NaNs.csv")
tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-whitened-using-PCA-with-dummy-vars-med-imput-NaNs.csv")
# y
y_path=os.path.join( os.getcwd(), "..", "data", "y-labels.csv")


#**************
# no NaNs
# tX
#tX_path=os.path.join( os.getcwd(), "..", "data", "training-data-standardized-with-dummy-vars-no-NaNs-removed-corr.csv")
# y
#y_path=os.path.join( os.getcwd(), "..", "data", "y-labels-no-NaNs.csv")

tX = np.loadtxt(tX_path)
y = np.loadtxt(y_path)

In [127]:
N_SAMPLES = tX.shape[0]
print(N_SAMPLES)
print(y.shape[0])
print(tX.shape)

250000
250000
(250000, 31)


In [128]:
y_colors = np.array(['b']*N_SAMPLES)
y_colors[y==0] = 'r'
print("Found {b} boson events out of {N} total events".format(b=np.sum(y), N=N_SAMPLES))
# If I understood correctly, 1 are bosons, i.e. boson events will be colored in blue

Found 85667.0 boson events out of 250000 total events


## Compute Polynomial Basis

In [129]:
def full_multivar_poly_basis_deg_two(x):
    n_feat = x.shape[1]
    n_samp = x.shape[0]

    #do a first iteration to avoid concatenating empty array problems
    temp0 = x[0,:]
    temp = x[0,:]
    temp = np.outer(temp0,temp)
    upper_tri_indices = np.triu_indices(n=temp.shape[0],m=temp.shape[1])
    temp = temp[upper_tri_indices]
        
    x_ret = temp
    
    for sample in range(1,n_samp):
        temp0 = x[sample,:].reshape(n_feat,1)
        temp = x[sample,:].reshape(n_feat,1)
#        print(temp.shape)
        temp = np.outer(temp0,temp)
        upper_tri_indices = np.triu_indices(n=temp.shape[0],m=temp.shape[1])
        temp = temp[upper_tri_indices]
        x_ret = np.r_[x_ret, temp]
        
    x_ret = x_ret.reshape(n_samp, int(x_ret.size/n_samp))
    return x_ret
        

In [130]:
def reduced_multivar_poly_basis_deg_n(x, n):
    if n < 3:
        return np.array([]).reshape(x.shape[0],0)
    return np.power(x,n)

In [131]:
def my_poly_basis(x,deg):
    assert(deg > 0)
    x_ret = np.c_[np.ones((x.shape[0],1)), x]
    if deg <= 1:
        return x_ret
    
    x_ret = np.c_[x_ret, full_multivar_poly_basis_deg_two(x)]
    
    if deg > 1 and deg <= 2:
        return x_ret
    
    for d in range(3,deg+1):
        x_ret = np.c_[x_ret, reduced_multivar_poly_basis_deg_n(x, d)]
        
    return x_ret

In [67]:
## test
a = np.array([2,5,11,7,9,13]).reshape((2,3))
print(a)
print("----")

my_poly_basis(a,3)

[[ 2  5 11]
 [ 7  9 13]]
----


array([[  1.00000000e+00,   2.00000000e+00,   5.00000000e+00,
          1.10000000e+01,   4.00000000e+00,   1.00000000e+01,
          2.20000000e+01,   2.50000000e+01,   5.50000000e+01,
          1.21000000e+02,   8.00000000e+00,   1.25000000e+02,
          1.33100000e+03],
       [  1.00000000e+00,   7.00000000e+00,   9.00000000e+00,
          1.30000000e+01,   4.90000000e+01,   6.30000000e+01,
          9.10000000e+01,   8.10000000e+01,   1.17000000e+02,
          1.69000000e+02,   3.43000000e+02,   7.29000000e+02,
          2.19700000e+03]])

In [137]:
#tX_poly = my_poly_basis(tX,5)
print(np.max(np.abs(tX)))
degree = 7
tX_poly = np.ones((tX.shape[0],1))
for deg in range(1,degree+1):
    tX_poly = np.c_[tX_poly, np.power(tX,deg)]

1.0


In [138]:
print(np.max(np.abs(tX_poly)))
print(tX_poly.shape)

1.0
(250000, 218)


# Logistic Regression

In [139]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return 1/(1+np.exp(-t))

In [183]:
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    Xw = tx.dot(w)
#    print(Xw)
#    print(y.shape)
    LOG_part = np.log( 1 + np.exp( Xw ) )
#    print(LOG_part.shape)
    PROD_part = np.multiply(y.reshape(N_SAMPLES,1), Xw )
#    print(PROD_part.shape)
    a = LOG_part - PROD_part
    return np.sum( a )

In [184]:
def calculate_gradient(y, tx, w,lambda_=0.):
    """compute the gradient of loss."""
    return tx.T.dot( sigmoid(tx.dot(w)) - y.reshape((tx.shape[0],1)) ) # - lambda_*w

## Gradient Descent

In [ ]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """

    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y,tx, w)
    w -= gamma*grad
    
    return loss, w

In [ ]:
from helpers import de_standardize
from plots import visualization

def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    gamma = 0.001
    losses = []

    # build tx
    tx = np.c_[.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return losses, w

In [ ]:
losses, w_star = logistic_regression_gradient_descent_demo(y, whit_tX)
#print(w_star.shape)
#print(w_star)

In [ ]:
f = plt.figure()
ax=f.add_subplot(2,1,1)
ax.scatter(whit_tX[:,0],whit_tX[:,1], c=y_colors, alpha=0.1)

ax=f.add_subplot(2,1,2)
prediction = sigmoid(whit_tX[:,0:1].dot(w_star[1:2]))
prediction = prediction < 0.5
#ax.plot(prediction)
ax.scatter(whit_tX[:,0],whit_tX[:,1], c=prediction, alpha=0.1)



In [ ]:
error = np.sum(np.abs(prediction - y.reshape(N_SAMPLES,1)))
print(error)
print(error/N_SAMPLES)

## Newton's Method

In [185]:
def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    S1 = sigmoid(tx.dot(w))
    S1 = S1.reshape((N_SAMPLES,1))
#    print(S1.shape)
    S2 = 1.0 - sigmoid(tx.dot(w))
#    print(S2.shape)
    S = np.multiply(S1,S2)
#    print(S.shape)
    S = scipy.sparse.spdiags(S[:,0], 0, N_SAMPLES, N_SAMPLES)
    return tx.T.dot(S.dot(tx))

In [186]:
def logistic_regression(y, tx, w,lambda_):
    """return the loss, gradient, and hessian."""
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y,tx, w,lambda_)
    hess = calculate_hessian(y, tx, w)
    return loss, grad, hess

In [187]:
def learning_by_newton_method(y, tx, w, gamma, lambda_):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, grad, hess = logistic_regression(y,tx,w,lambda_)
    w -= gamma * np.linalg.inv(hess).dot(grad);
    return loss, w

In [203]:
def logistic_regression_newton_method_demo(y, tx, w_initial):
    # init parameters
    max_iter = 5000
    gamma = 0.2e-3
    threshold = 1e-8
    lambda_ = 0.1
    losses = []

    # build tx
    #tx = np.c_[np.ones((y.shape[0], 1)), x] #CAREFUL: poly basis already puts in the ones
    w = w_initial

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma, lambda_)
        # log info
        if iter % 25 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2])/np.abs(losses[-1]) < threshold:
            print("[Exit Condition Met]: Current iteration={i}, the loss={l}".format(i=iter, l=loss)) 
            break
    return losses, w

In [204]:
#w0 = np.random.rand(tX_poly.shape[1]+1,1)
w0 = np.random.rand(tX_poly.shape[1],1)
losses, w_star = logistic_regression_newton_method_demo(y, tX_poly, w0)

Current iteration=0, the loss=215738.95177430558
Current iteration=25, the loss=214467.69967754293
Current iteration=50, the loss=213231.75735038
Current iteration=75, the loss=212026.5644230777
Current iteration=100, the loss=210849.51758413753
Current iteration=125, the loss=209698.65168626318
Current iteration=150, the loss=208572.36993077368
Current iteration=175, the loss=207469.3256492951
Current iteration=200, the loss=206388.35513562823
Current iteration=225, the loss=205328.4345026383
Current iteration=250, the loss=204288.65007070877
Current iteration=275, the loss=203268.17716684073
Current iteration=300, the loss=202266.26447017788
Current iteration=325, the loss=201282.22216529297
Current iteration=350, the loss=200315.4127880863
Current iteration=375, the loss=199365.2440221882
Current iteration=400, the loss=198431.1629365646
Current iteration=425, the loss=197512.6513070036
Current iteration=450, the loss=196609.22176563853
Current iteration=475, the loss=195720.4145920

/home/checco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: overflow encountered in exp
/home/checco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: RuntimeWarning: invalid value encountered in double_scalars


Current iteration=2100, the loss=inf
Current iteration=2125, the loss=inf
Current iteration=2150, the loss=inf
Current iteration=2175, the loss=inf
Current iteration=2200, the loss=inf
Current iteration=2225, the loss=inf
Current iteration=2250, the loss=inf
Current iteration=2275, the loss=inf
Current iteration=2300, the loss=inf
Current iteration=2325, the loss=inf
Current iteration=2350, the loss=inf
Current iteration=2375, the loss=inf
Current iteration=2400, the loss=inf
Current iteration=2425, the loss=inf
Current iteration=2450, the loss=inf
Current iteration=2475, the loss=inf
Current iteration=2500, the loss=inf
Current iteration=2525, the loss=inf
Current iteration=2550, the loss=inf
Current iteration=2575, the loss=inf
Current iteration=2600, the loss=inf
Current iteration=2625, the loss=inf
Current iteration=2650, the loss=inf
Current iteration=2675, the loss=inf
Current iteration=2700, the loss=inf
Current iteration=2725, the loss=inf
Current iteration=2750, the loss=inf
C

/home/checco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()


Current iteration=3050, the loss=inf
Current iteration=3075, the loss=inf
Current iteration=3100, the loss=inf
Current iteration=3125, the loss=inf
Current iteration=3150, the loss=inf
Current iteration=3175, the loss=inf
Current iteration=3200, the loss=inf
Current iteration=3225, the loss=inf
Current iteration=3250, the loss=inf
Current iteration=3275, the loss=inf
Current iteration=3300, the loss=inf
Current iteration=3325, the loss=inf
Current iteration=3350, the loss=inf
Current iteration=3375, the loss=inf
Current iteration=3400, the loss=inf
Current iteration=3425, the loss=inf
Current iteration=3450, the loss=inf
Current iteration=3475, the loss=inf
Current iteration=3500, the loss=inf
Current iteration=3525, the loss=inf
Current iteration=3550, the loss=inf
Current iteration=3575, the loss=inf
Current iteration=3600, the loss=inf
Current iteration=3625, the loss=inf
Current iteration=3650, the loss=inf
Current iteration=3675, the loss=inf
Current iteration=3700, the loss=inf
C

## Compute Predictions

In [190]:
#tX_pred = np.c_[np.ones((y.shape[0], 1)), tX]
tX_pred = tX_poly
prediction = sigmoid(tX_pred.dot(w_star))
prediction = np.array( [ int(x) for x in (prediction > 0.5)] )

## Plot predictions

In [ ]:
f = plt.figure()
ax=f.add_subplot(2,2,1)
ax.scatter(tX[:,0],tX[:,1], c=y_colors, alpha=0.1)

ax=f.add_subplot(2,2,2)
ax.scatter(tX[:,2],tX[:,3], c=y_colors, alpha=0.1)

prediction_colors = np.array(['b']*N_SAMPLES)
prediction_colors[prediction==0] = 'r'

ax=f.add_subplot(2,2,3)
ax.scatter(tX[:,0],tX[:,1], c=prediction_colors, alpha=0.1)

ax=f.add_subplot(2,2,4)
ax.scatter(tX[:,2],tX[:,3], c=prediction_colors, alpha=0.1)

## Compute training error

In [191]:
error = np.sum(np.abs(prediction - y))
print(prediction)
print(y)
print(error)
print(error/N_SAMPLES)

[0 0 0 ..., 0 0 0]
[ 1.  0.  0. ...,  1.  0.  0.]
49238.0
0.196952
